# 1. 패키지 설치

In [2]:
%pip install langchain langchain-core langchain-community langchain-text-splitters langchain-openai langchain-pinecone


[notice] A new release of pip is available: 23.0.1 -> 24.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


# 2. Knowledge Base 구성을 위한 데이터 생성

- [3.4 LangChain을 활용한 Vector Database 변경 (Chroma ➡️ Pinecone)](https://github.com/jasonkang14/inflearn-rag-notebook/blob/main/3.4%20LangChain%EC%9D%84%20%ED%99%9C%EC%9A%A9%ED%95%9C%20Vector%20Database%20%EB%B3%80%EA%B2%BD%20(Chroma%20%E2%9E%A1%EF%B8%8F%20Pinecone).ipynb)와 동일함

In [3]:
from langchain_community.document_loaders import Docx2txtLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1500,
    chunk_overlap=200,
)

loader = Docx2txtLoader('./tax_with_markdown.docx')
document_list = loader.load_and_split(text_splitter=text_splitter)

In [4]:
document_list[52]

Document(page_content='제55조(세율) ①거주자의 종합소득에 대한 소득세는 해당 연도의 종합소득과세표준에 다음의 세율을 적용하여 계산한 금액(이하 “종합소득산출세액”이라 한다)을 그 세액으로 한다. <개정 2014. 1. 1., 2016. 12. 20., 2017. 12. 19., 2020. 12. 29., 2022. 12. 31.>\n\n| 종합소득 과세표준          | 세율                                         |\n\n|-------------------|--------------------------------------------|\n\n| 1,400만원 이하     | 과세표준의 6퍼센트                             |\n\n| 1,400만원 초과     5,000만원 이하     | 84만원 + (1,400만원을 초과하는 금액의 15퍼센트)  |\n\n| 5,000만원 초과   8,800만원 이하     | 624만원 + (5,000만원을 초과하는 금액의 24퍼센트) |\n\n| 8,800만원 초과 1억5천만원 이하    | 3,706만원 + (8,800만원을 초과하는 금액의 35퍼센트)|\n\n| 1억5천만원 초과 3억원 이하         | 3,706만원 + (1억5천만원을 초과하는 금액의 38퍼센트)|\n\n| 3억원 초과    5억원 이하         | 9,406만원 + (3억원을 초과하는 금액의 38퍼센트)   |\n\n| 5억원 초과      10억원 이하        | 1억 7,406만원 + (5억원을 초과하는 금액의 42퍼센트)|\n\n| 10억원 초과        | 3억 8,406만원 + (10억원을 초과하는 금액의 45퍼센트)|\n\n\n\n\n\n② 거주자의 퇴직소득에 대한 소득세는 다음 각 호의 순서에 따라 계산한 금액(이하 “퇴직소득 산출세액”이라 한다)으로 한다.<개정 2013. 1. 1., 2014. 12. 23.>\n\n1. 해당 

In [5]:
from dotenv import load_dotenv
from langchain_openai import OpenAIEmbeddings

load_dotenv()

embedding = OpenAIEmbeddings(model='text-embedding-3-large')

In [6]:
import os

from pinecone import Pinecone
from langchain_pinecone import PineconeVectorStore

index_name = 'tax-markdown-index'
pinecone_api_key = os.environ.get("PINECONE_API_KEY")
pc = Pinecone(api_key=pinecone_api_key)

# 데이터를 추가할 때
database = PineconeVectorStore.from_documents(document_list, embedding, index_name=index_name)

# 이미 생성된 데이터베이스를 사용할 때 
# database = PineconeVectorStore.from_existing_index(index_name=index_name, embedding=embedding)


/Users/jasonkang/.pyenv/versions/3.10.12/envs/inflearn-llm-application/lib/python3.10/site-packages/pinecone/data/index.py:1: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


In [7]:
query = '연봉 5천만원인 거주자의 종합소득세는?'

# 3. 답변 생성을 위한 Retrieval

- `RetrievalQA`에 전달하기 위해 `retriever` 생성
- `search_kwargs` 의 `k` 값을 변경해서 가져올 문서의 갯수를 지정할 수 있음
- `.invoke()` 를 호출해서 어떤 문서를 가져오는지 확인 가능

In [8]:
retriever = database.as_retriever(search_kwargs={'k': 4})
retriever.invoke(query)

[Document(page_content='제55조(세율) ①거주자의 종합소득에 대한 소득세는 해당 연도의 종합소득과세표준에 다음의 세율을 적용하여 계산한 금액(이하 “종합소득산출세액”이라 한다)을 그 세액으로 한다. <개정 2014. 1. 1., 2016. 12. 20., 2017. 12. 19., 2020. 12. 29., 2022. 12. 31.>\n\n| 종합소득 과세표준          | 세율                                         |\n\n|-------------------|--------------------------------------------|\n\n| 1,400만원 이하     | 과세표준의 6퍼센트                             |\n\n| 1,400만원 초과     5,000만원 이하     | 84만원 + (1,400만원을 초과하는 금액의 15퍼센트)  |\n\n| 5,000만원 초과   8,800만원 이하     | 624만원 + (5,000만원을 초과하는 금액의 24퍼센트) |\n\n| 8,800만원 초과 1억5천만원 이하    | 3,706만원 + (8,800만원을 초과하는 금액의 35퍼센트)|\n\n| 1억5천만원 초과 3억원 이하         | 3,706만원 + (1억5천만원을 초과하는 금액의 38퍼센트)|\n\n| 3억원 초과    5억원 이하         | 9,406만원 + (3억원을 초과하는 금액의 38퍼센트)   |\n\n| 5억원 초과      10억원 이하        | 1억 7,406만원 + (5억원을 초과하는 금액의 42퍼센트)|\n\n| 10억원 초과        | 3억 8,406만원 + (10억원을 초과하는 금액의 45퍼센트)|\n\n\n\n\n\n② 거주자의 퇴직소득에 대한 소득세는 다음 각 호의 순서에 따라 계산한 금액(이하 “퇴직소득 산출세액”이라 한다)으로 한다.<개정 2013. 1. 1., 2014. 12. 23.>\n\n1. 해당

# 4. Augmentation을 위한 Prompt 활용

- Retrieval된 데이터는 LangChain에서 제공하는 프롬프트(`"rlm/rag-prompt"`) 사용

In [9]:
from langchain import hub

prompt = hub.pull("rlm/rag-prompt")

# 5. 답변 생성

- [RetrievalQA](https://docs.smith.langchain.com/old/cookbook/hub-examples/retrieval-qa-chain)를 통해 LLM에 전달
    - `RetrievalQA`는 [create_retrieval_chain](https://python.langchain.com/v0.2/docs/how_to/qa_sources/#using-create_retrieval_chain)으로 대체됨
    - 실제 ChatBot 구현 시 `create_retrieval_chain`으로 변경하는 과정을 볼 수 있음
- 하단의 `dictionary_chain` 과 연계하여 사용

In [10]:

from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model='gpt-4o')

In [11]:
from langchain.chains import RetrievalQA


qa_chain = RetrievalQA.from_chain_type(
    llm, 
    retriever=retriever,
    chain_type_kwargs={"prompt": prompt}
)

In [12]:
ai_message = qa_chain.invoke({"query": query})

In [13]:
ai_message

{'query': '연봉 5천만원인 거주자의 종합소득세는?', 'result': '연봉 5천만원인 거주자의 종합소득세는 624만원입니다.'}

# 6. Retrieval을 위한 keyword 사전 활용

- Knowledge Base에서 사용되는 keyword를 활용하여 사용자 질문 수정
- LangChain Expression Language (LCEL)을 활용한 Chain 연계

In [14]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate

dictionary = ["사람을 나타내는 표현 -> 거주자"]

prompt = ChatPromptTemplate.from_template(f"""
    사용자의 질문을 보고, 우리의 사전을 참고해서 사용자의 질문을 변경해주세요.
    만약 변경할 필요가 없다고 판단된다면, 사용자의 질문을 변경하지 않아도 됩니다.
    그런 경우에는 질문만 리턴해주세요
    사전: {dictionary}
    
    질문: {{question}}
""")

dictionary_chain = prompt | llm | StrOutputParser()
tax_chain = {"query": dictionary_chain} | qa_chain

In [15]:
new_question = dictionary_chain.invoke({"question": query})

In [16]:
new_question

'연봉 5천만원인 거주자의 종합소득세는?'

In [17]:
ai_response = tax_chain.invoke({"question": query})

In [18]:
ai_response

{'query': '연봉 5천만원인 거주자의 종합소득세는?', 'result': '연봉 5천만원인 거주자의 종합소득세는 624만원입니다.'}